__<h1><center>Time Series Modelling</center></h1>__

## __Introduction__

The dataset that we have considered is based on World War 2. Here we use multiple data sources that are ___Aerial Bombing Operations___ and ___Weather Conditions___ in World War 2. To understand the data better we mainly follow __Exploratory Data Analysis__ approach where Data Description, Data Cleaning, Data Visualization etc are explored. The main focus is to do time series prediction to predict when bombing operations are done.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.plotly as py
import datetime as dt

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore")

plt.style.use('fivethirtyeight')

### __Load the Data__: Reading and Exploring the data.
- Aerial Bombing Operations during World War 2
- Weather Conditions during World War 2

In [2]:
aerial = pd.read_csv('operations.csv')
weather_location = pd.read_csv('weather_location.csv')
weather_summary = pd.read_csv('summary_weather.csv')

FileNotFoundError: File b'weather_location.csv' does not exist

### __Data Description__: Explaining the features of the data.
- Aerial Bombing Operations during World War 2
- Weather Condition during World War 2

### __Data Cleaning__: Dropping the NaN values from the data for Visualization purpose.

In [ ]:
aerial = aerial[pd.isna(aerial.Country)==False]
aerial = aerial[pd.isna(aerial['Target Longitude'])==False]
aerial = aerial[pd.isna(aerial['Takeoff Longitude'])==False]

drop_list = ['Mission ID','Unit ID','Target ID','Altitude (Hundreds of Feet)','Airborne Aircraft',
             'Attacking Aircraft','Bombing Aircraft','Aircraft Returned','Aircraft Failed',
             'Aircraft Damaged','Aircraft Lost','High Explosives', 'High Explosives Type',
             'Mission Type','High Explosives Weight (Pounds)','High Explosives Weight (Tons)',
             'Incendiary Devices','Incendiary Devices Type','Incendiary Devices Weight (Pounds)',
             'Incendiary Devices Weight (Tons)','Fragmentation Devices','Fragmentation Devices Type',
             'Fragmentation Devices Weight (Pounds)','Fragmentation Devices Weight (Tons)',
             'Total Weight (Pounds)','Total Weight (Tons)','Time Over Target','Bomb Damage Assessment',
             'Source ID']

aerial.drop(drop_list, axis=1,inplace = True)
aerial = aerial[ aerial.iloc[:,8]!="4248"]
aerial = aerial[ aerial.iloc[:,9]!=1355]

In [ ]:
weather_summary = weather_summary.loc[:,["STA","Date","MeanTemp"]]
weather_location = weather_location.loc[:,["WBAN","NAME","STATE/COUNTRY ID","Latitude","Longitude"]]

### __Data Visualization__: Basics of visualization to understand the data.

In [ ]:
print(aerial['Country'].value_counts())
plt.figure(figsize=(20, 10))
sns.countplot(aerial['Country'])
plt.title('Countries')
plt.show()

In [ ]:
# Top Target Countries
print(aerial['Target Country'].value_counts()[:10])
plt.figure(figsize=(20, 10))
sns.countplot(aerial['Target Country'])
plt.xticks(rotation=90)
plt.title('Target Countries')
plt.show()

In [ ]:
# Aircraft Series
aircraft = aerial['Aircraft Series'].value_counts()[:10]
print(aircraft[:10])
plt.figure(figsize=(20, 10))
sns.countplot(aerial['Aircraft Series'])
plt.title('Aircraft Series')
plt.xticks(rotation=90)
plt.show()

- Now we will focus on USA and BURMA war.
- In this war USA bomb BURMA (Katha city) from 1942 to 1945.
- The closest weather station to this war is __BINDUKURI__ and it has temperature record from 1943 to 1945.

In [ ]:
weather_station_id = weather_location[weather_location.NAME == 'BINDUKARI'].WBAN
weather_bin = weather_summary[weather_summary.STA == 32907]
weather_bin["Date"] = pd.to_datetime(weather_bin["Date"])
plt.figure(figsize=(20 ,10))
plt.plot(weather_bin.Date, weather_bin.MeanTemp)
plt.title("Mean Temperature of Bindukuri Area")
plt.xlabel("Date")
plt.ylabel("Mean Temperature")
plt.show()

### __Observation__
- As observed we can see temperature oscillates between 12 and 32 degrees.
- Temperature of winter months is colder than temperature of summer months.

In [ ]:
aerial = pd.read_csv('operations.csv')
aerial['year'] = [each.split('/')[2] for each in aerial['Mission Date']]
aerial['month'] = [each.split('/')[0] for each in aerial['Mission Date']]
aerial = aerial[aerial['year'] >= '1943']
aerial = aerial[aerial['month'] >= '8']

aerial['Mission Date'] = pd.to_datetime(aerial['Mission Date'])

attack = 'USA'
target = 'BURMA'
city = 'KATHA'

aerial_war = aerial[aerial.Country == attack]
aerial_war = aerial_war[aerial_war["Target Country"] == target]
aerial_war = aerial_war[aerial_war["Target City"] == city]

### __Prediction__: Time Series Prediction with ARIMA.

* $ARIMA$: AutoRegressive Integrated Moving Average

### __Basic Milestones__
 * To check for Stationarity of the Time Series data.<br>
 * To obtain Dickey-Fuller Test and compare the test statistic with the critical value.<br>
 * To obtain ACF and PACF plots and measure the correlation between time series and lagged version.<br>
 * To obtain the predictions and forecast the future values and reduce the error factor.

### __Stationarity of Time Series__

- There are three basic criterion for a time series to understand whether it is stationary series or not.
    - Statistical properties of time series such as mean, variance should remain constant over time.
        - Constant Mean
        - Constant Variance
        - Auto-covariance should not depend on time (covariance between time series and lagged time series).

In [ ]:
# lets create time series from weather
timeSeries = weather_bin.loc[:, ['Date', 'MeanTemp']]
timeSeries.index = timeSeries.Date
ts = timeSeries.drop('Date', axis=1)

We check stationarity of time series using the following methods
- __Plotting Rolling Statistics__: Suppose if we have a window lets say window size is 6 and we find rolling mean and rolling variance to check stationary.
- __Dickey-Fuller Test__: The test results comprise of a Test Statistic and some Critical Values for difference confidence levels. If the test statistic is less than the critical value, we can say that time series is stationary.

In [ ]:
def check_adfuller(ts):
    # Dickey-Fuller test
    result = adfuller(ts, autolag='AIC')
    print('Test statistic: ' , result[0])
    print('p-value: '  ,result[1])
    print('Critical Values:' ,result[4])

def check_mean_std(ts):
    # Rolling statistics
    rolmean = ts.rolling(window=6).mean()
    rolstd = ts.rolling(window=6).std()
    plt.figure(figsize=(20, 10))   
    orig = plt.plot(ts, color='red', label='Original')
    mean = plt.plot(rolmean, color='black', label='Rolling Mean')
    std = plt.plot(rolstd, color='green', label='Rolling Std')
    plt.xlabel("Date")
    plt.ylabel("Mean Temperature")
    plt.title('Rolling Mean & Standard Deviation')
    plt.legend()
    plt.show()

# check stationary: mean, variance(std) and adfuller test
check_mean_std(ts)
check_adfuller(ts.MeanTemp)

### __Observation__

From the above plot we can infer that:
- Mean is not constant $\rightarrow$ non-stationary
- Variance is contant $\rightarrow$ stationary
- Test statistic $>$ critical value $\rightarrow$ non-stationary

Therefore, time series is not stationary

### __Making the time series data stationary__

- There are mainly two reasons behind non-stationarity of time series
    - __Trend__: Mean varies over time.
    - __Seasonality__: Variations at specific time. Constant variance is required.
- Stationarity can be acheived by
    - __Differencing method__: This method is one of the most common methods. The idea is that we take the difference between time series and shifted time series.

In [ ]:
# differencing method
ts_diff = ts - ts.shift()
plt.figure(figsize=(20, 10))
plt.plot(ts_diff)
plt.title("Differencing method") 
plt.xlabel("Date")
plt.ylabel("Differencing Mean Temperature")
plt.show()

From the above plot we infer that the mean is constant.

In [ ]:
ts_diff.dropna(inplace=True) # due to shifting there are nan values

# check stationary: mean, variance(std) and adfuller test
check_mean_std(ts_diff)
check_adfuller(ts_diff.MeanTemp)

### __Observation__

- __Constant mean criteria__: Black line indicates that the mean is constant $\rightarrow$ stationarity
- __Constant variance criteria__: Green line indicates that the variance is constant $\rightarrow$ stationarity
- Test statistic is smaller that $1\%$ critical value, so the series is stationary.

### __Forecasting a time series__
- For prediction(forecasting) we will use ts_diff time series that is result of differencing method.
- Also prediction method is ARIMA that is Auto-Regressive Integrated Moving Averages.
    - $AR$: Auto-Regressive ($p$): $AR$ terms are just lags of dependent variable. For example if we take $$p = 3$$ we will use $x(t-1)$, $x(t-2)$ and $x(t-3)$ to predict $x(t)$.
    - $I$: Integrated ($d$): These are the number of non-seasonal differences. For example, in our case we take the first order difference. So we pass that variable and put $$d = 0$$
    - $MA$: Moving Averages ($q$): $MA$ terms are lagged forecast errors in prediction equation.
- In $ARIMA$ we use $(p, d, q)$ as parameters of the model. We choose these values from two different plots.
    - $ACF$: Measurement of the correlation between time series and lagged version of time series.
    - $PACF$: Measurement of correlation between the time series and lagged version of time series after eliminatind the variations that have already compared.

In [ ]:
# ACF and PACF
lag_acf = acf(ts_diff, nlags=20)
lag_pacf = pacf(ts_diff, nlags=20, method='ols')

# ACF
plt.figure(figsize=(20, 10))
plt.subplot(121) 
plt.plot(lag_acf)
plt.axhline(y=0, linestyle='--', color='gray')
plt.axhline(y=-1.96/np.sqrt(len(ts_diff)), linestyle='--', color='gray')
plt.axhline(y=1.96/np.sqrt(len(ts_diff)), linestyle='--', color='gray')
plt.title('Autocorrelation Function')

# PACF
plt.subplot(122)
plt.plot(lag_pacf)
plt.axhline(y=0, linestyle='--', color='gray')
plt.axhline(y=-1.96/np.sqrt(len(ts_diff)), linestyle='--', color='gray')
plt.axhline(y=1.96/np.sqrt(len(ts_diff)), linestyle='--', color='gray')
plt.title('Partial Autocorrelation Function')

plt.tight_layout()
plt.show()

### __Observation__

- From the above plots two dotted lines are the confidence interevals. We use these lines to determine the $p$ and $q$ values
    - __Choosing $p$__: The lag value where the PACF chart crosses the upper confidence interval for the first time. $$p = 1$$
    - __Choosing $q$__: The lag value where the ACF chart crosses the upper confidence interval for the first time. $$q = 1$$
- We will use $(1, 0, 1)$ as parameters of $ARIMA$ model and do the predictions.

In [ ]:
# fit the model
model = ARIMA(ts, order=(1,0,1)) # (ARMA) = (1,0,1)
model_fit = model.fit(disp=0)

# predict
start_index = dt.datetime(1944, 6, 25)
end_index = dt.datetime(1945, 5, 31)
forecast = model_fit.predict(start=650, end=750)

plt.figure(figsize=(20, 10))
plt.plot(weather_bin.Date, weather_bin.MeanTemp, label="original")
plt.plot(forecast, label="predicted")
plt.title("Time Series Forecast")
plt.xlabel("Date")
plt.ylabel("Mean Temperature")
plt.legend()
plt.show()

### __Mean Squared Error Calculation__

In [ ]:
model2 = ARIMA(ts, order=(1,0,1)) # (ARMA) = (1,0,1)
model_fit2 = model2.fit(disp=0)
forecast2 = model_fit2.predict()
error = mean_squared_error(ts, forecast2)
print("error: " ,error)

plt.figure(figsize=(20, 10))
plt.plot(weather_bin.Date, weather_bin.MeanTemp, label="original")
plt.plot(forecast2, label="predicted")
plt.title("Time Series Forecast")
plt.xlabel("Date")
plt.ylabel("Mean Temperature")
plt.legend()
plt.show()

### __Observation__

The red curve represents the plot of fitted / predicted values and the black curve represents the plot of observed / original values. Both the plots lie close to each other. This shows that the fitted model has very less error component.

### __Conclusion__

As the introduction suggests the dataset that we have considerd is based on World War 2. Our main focus was to predict and forecast when the next bombing takes place. We have done the forecasting using ARIMA modelling technique and predicted the future values. The model shows less error factor which suggests that the applied model is good. <br> Working with this data we have learnt the preliminary aspects of Data Science and further we would like work on different datasets to get hands on and gain experiance in the field of Data Science.